# Boson Sampling

One notable quantum advantage scheme that can be implemented on a photonic quantum computer is the well-known Boson Sampling scheme proposed by Aaronson and Arkhipov <cite data-footcite="aaronson:2010"></cite>. Boson Sampling is a type of photonic quantum computation that starts from Fock basis states and evolves the photonic quantum states by an interferometer. At the end of the Boson Sampling circuit, particle number measurements are carried out to generate samples. Although this setup is theoretically applicable to any bosonic system, the photonic implementation is the most natural. It is generally accepted that Boson Sampling executes a computational task more efficiently than classical computers because solving it requires sampling from the output particle number probability distribution with a randomly chosen interferometer, a challenge considered intractable for classical computation.

In the standard Boson Sampling setup, the input state is typically a Fock basis state $\ket{ S } = | S_1, S_2, \ldots S_d \rangle$, where $n=\sum_{i=1}^d S_i$ denotes the total number of photons. Although $\ket{ S }$ can be arbitrary, the values of $S_i$ are typically chosen to be either $1$ or $0$. The photons are then passed through a generic passive linear optical interferometer, characterized by a $d \times d$ unitary matrix $U$. Since a passive linear circuit preserves the total number of particles, the state vector remains within the $n$-particle subspace of dimension $\binom{n+d-1}{d-1}$ throughout the entire evolution. Finally, a particle number measurement is performed on each mode, and the probability of obtaining a specific measurement outcome $\ket{T} = \ket{T_1, T_2, \ldots, T_d}$ is given by
$$
    p(S, T) =
    \frac{
        |
        \operatorname{per}
            ( \operatorname{red}_{T, S} (U) )
        |^2
    }{
        S_1! \dots S_d! T_1! \dots T_d!
    },
$$
where $\operatorname{red}_{T, S}$ is the reduction defined as repeating the rows and columns according to $T$ and $S$, respectively. $U$ is the unitary matrix corresponding to the passive linear circuit, and the permanent $\operatorname{per}$ of a matrix is defined via
$$
    \operatorname{per}(A) = \sum_{\sigma \in S_n}  \prod_{i = 1}^n A_{\sigma(i), i}, \qquad A \in \mathbb{C}^{n \times n}.
$$
Most importantly, the permanent is the source of the widely accepted classical intractability of the Boson Sampling problem: given a general matrix $A \in \mathbb{C}^{n \times n}$ even approximating $\operatorname{per}(A)$ is $\#P$-complete, and finding a polynomial-time algorithm for calculating this would imply P = NP <cite data-footcite="aaronson:2010"></cite>.

If you want to run the Boson Sampling algorithm in Piquasso, it is easy to do by using [SamplingSimulator](../simulators/sampling.rst#piquasso._simulators.sampling.simulator.SamplingSimulator) with the [Interferometer](../instructions/gates.rst#piquasso.instructions.gates.Interferometer) instruction. A simple example is given as follows:

In [3]:
import piquasso as pq

from scipy.stats import unitary_group

d = 7

interferometer_matrix = unitary_group.rvs(d)

with pq.Program() as program:
    pq.Q(all) | pq.StateVector([1, 1, 1, 0, 0, 0, 0])

    pq.Q(all) | pq.Interferometer(interferometer_matrix)

    pq.Q(all) | pq.ParticleNumberMeasurement()

simulator = pq.SamplingSimulator(d=d)

result = simulator.execute(program, shots=20)

print(result.samples)

[(1, 0, 0, 0, 1, 1, 0), (0, 1, 0, 0, 2, 0, 0), (1, 0, 1, 0, 1, 0, 0), (0, 0, 2, 0, 1, 0, 0), (0, 1, 0, 1, 0, 0, 1), (0, 1, 0, 2, 0, 0, 0), (0, 0, 0, 1, 0, 0, 2), (0, 0, 1, 0, 0, 1, 1), (1, 1, 0, 1, 0, 0, 0), (0, 0, 0, 1, 1, 1, 0), (0, 1, 0, 0, 1, 1, 0), (0, 0, 0, 2, 0, 0, 1), (0, 1, 0, 0, 1, 1, 0), (0, 0, 0, 0, 3, 0, 0), (0, 1, 0, 0, 0, 2, 0), (0, 0, 2, 0, 0, 0, 1), (0, 1, 0, 0, 1, 0, 1), (0, 0, 0, 2, 1, 0, 0), (0, 0, 0, 1, 2, 0, 0), (0, 0, 0, 2, 0, 0, 1)]


If you want to execute lossy Boson Sampling, you can do this by using the [Loss](../instructions/channels.rst#piquasso.instructions.channels.Loss) channel:

In [4]:
transmissivity = 0.9

with pq.Program() as program:
    pq.Q(all) | pq.StateVector([1, 1, 1, 0, 0, 0, 0])

    pq.Q(all) | pq.Interferometer(interferometer_matrix)

    for i in range(d):
        pq.Q(i) | pq.Loss(transmissivity)

    pq.Q(all) | pq.ParticleNumberMeasurement()

result = simulator.execute(program, shots=20)

print(result.samples)

[(0, 0, 0, 2, 0, 0, 1), (0, 1, 1, 0, 0, 0, 1), (0, 1, 0, 0, 0, 2, 0), (1, 0, 0, 0, 2, 0, 0), (0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 1, 1, 0), (0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 1, 2, 0, 0), (0, 2, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 1, 0), (0, 0, 2, 0, 0, 1, 0), (0, 0, 0, 1, 1, 1, 0), (1, 0, 0, 0, 0, 1, 1), (0, 2, 0, 0, 1, 0, 0), (0, 0, 1, 1, 0, 1, 0), (0, 0, 0, 0, 0, 2, 0), (0, 1, 1, 0, 1, 0, 0), (0, 0, 0, 1, 2, 0, 0), (0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 1)]
